# Лабораторная работа 7. Управление ресурсами в однопроцессорной системе с неоднородными заявками
## Задание 1

In [16]:
Variant <- 5
set.seed(Variant)
m <- sample(c(6:20), 1)
lambda <- runif(1, 0.1, 2)
Q <- rexp(m, 0.3)
q <- sample(c(1:4), 1)
View(data.frame(m, q, lambda))
print(Q)

m,q,lambda
<int>,<int>,<dbl>
7,1,1.401915


[1] 0.2417931 1.3403831 0.1863999 2.0529013 0.2552007 2.9172096 6.6709868


In [29]:
N <- 10000
# N <- 10
programs <- sample(Q, N, replace = TRUE)
programs

[1] 0.2417931 0.2417931 1.3403831 2.0529013 2.9172096 2.9172096 0.2552007
    [8] 6.6709868 2.0529013 2.9172096 1.3403831 0.2552007 0.1863999 2.9172096
   [15] 2.0529013 1.3403831 2.9172096 0.2552007 2.9172096 6.6709868 6.6709868
   [22] 0.2552007 0.2552007 6.6709868 2.9172096 6.6709868 1.3403831 6.6709868
   [29] 2.0529013 0.2417931 1.3403831 0.2417931 1.3403831 0.2417931 0.2552007
   [36] 6.6709868 6.6709868 0.1863999 2.9172096 0.1863999 6.6709868 1.3403831
   [43] 0.1863999 0.1863999 2.9172096 2.0529013 2.9172096 6.6709868 0.2417931
   [50] 6.6709868 0.2417931 1.3403831 0.2552007 2.0529013 2.9172096 2.0529013
   [57] 6.6709868 2.0529013 6.6709868 2.9172096 0.1863999 0.2552007 1.3403831
   [64] 0.2552007 0.1863999 0.2552007 6.6709868 2.9172096 0.2417931 0.2552007
   [71] 2.0529013 0.1863999 0.2552007 1.3403831 0.2417931 2.0529013 1.3403831
   [78] 2.0529013 0.2417931 0.1863999 0.2552007 1.3403831 0.2417931 0.2552007
   [85] 6.6709868 0.2552007 2.9172096 0.2552007 0.1863999 0.2552007 0.1863999
   [92] 2.9172096 0.2417931 6.6709868 6.6709868 0.2417931 0.2417931 1.3403831
   [99] 0.2552007 2.9172096 6.6709868 2.0529013 1.3403831 0.1863999 0.2552007
  [106] 0.1863999 0.1863999 2.0529013 1.3403831 1.3403831 0.2552007 2.0529013
  [113] 2.9172096 1.3403831 0.2417931 1.3403831 1.3403831 2.9172096 2.9172096
  [120] 2.0529013 2.9172096 2.0529013 2.9172096 0.2552007 0.1863999 1.3403831
  [127] 0.1863999 6.6709868 1.3403831 0.2417931 1.3403831 6.6709868 0.2417931
  [134] 6.6709868 1.3403831 2.9172096 0.2417931 1.3403831 1.3403831 0.2417931
  [141] 2.9172096 0.1863999 2.0529013 6.6709868 2.9172096 0.2552007 1.3403831
  [148] 6.6709868 2.9172096 0.1863999 2.9172096 0.2417931 6.6709868 0.1863999
  [155] 2.0529013 0.1863999 2.0529013 6.6709868 0.1863999 2.0529013 1.3403831
  [162] 2.0529013 2.9172096 0.2552007 6.6709868 1.3403831 0.2552007 0.2552007
  [169] 0.1863999 1.3403831 0.2552007 0.2417931 2.9172096 1.3403831 0.2552007
  [176] 0.2417931 0.1863999 1.3403831 6.6709868 6.6709868 0.2417931 6.6709868
  [183] 0.2552007 0.1863999 0.2552007 2.0529013 0.2552007 2.0529013 1.3403831
  [190] 1.3403831 2.9172096 6.6709868 2.0529013 2.0529013 0.1863999 2.9172096
  [197] 6.6709868 1.3403831 6.6709868 6.6709868 2.0529013 6.6709868 0.1863999
  [204] 0.2417931 1.3403831 2.0529013 0.2552007 6.6709868 0.2417931 6.6709868
  [211] 0.1863999 0.2417931 6.6709868 0.2552007 6.6709868 2.0529013 0.2552007
  [218] 0.2552007 0.1863999 0.2552007 0.2417931 2.0529013 2.0529013 2.0529013
  [225] 0.1863999 0.1863999 0.2417931 0.2417931 0.2552007 0.2417931 2.9172096
  [232] 0.1863999 1.3403831 0.2417931 2.9172096 0.2417931 0.2552007 0.2417931
  [239] 0.2417931 0.2552007 0.2417931 6.6709868 0.2552007 0.2417931 0.1863999
  [246] 0.2552007 6.6709868 2.0529013 2.9172096 0.1863999 1.3403831 2.9172096
  [253] 2.0529013 6.6709868 6.6709868 1.3403831 6.6709868 0.2552007 0.2417931
  [260] 2.0529013 0.2552007 2.0529013 6.6709868 0.2417931 0.2552007 0.2552007
  [267] 0.2417931 0.2417931 0.2552007 0.1863999 1.3403831 6.6709868 0.2417931
  [274] 0.2552007 0.2552007 0.2552007 0.1863999 0.1863999 1.3403831 0.2417931
  [281] 6.6709868 1.3403831 6.6709868 6.6709868 0.2552007 1.3403831 6.6709868
  [288] 6.6709868 2.0529013 0.2552007 0.2417931 0.1863999 6.6709868 2.9172096
  [295] 6.6709868 0.2552007 2.0529013 2.9172096 2.0529013 0.2417931 6.6709868
  [302] 0.1863999 0.1863999 0.2552007 2.9172096 0.1863999 2.0529013 2.0529013
  [309] 6.6709868 0.2552007 0.2417931 2.9172096 2.9172096 0.2552007 6.6709868
  [316] 2.0529013 0.1863999 1.3403831 0.1863999 2.0529013 2.9172096 0.1863999
  [323] 0.2417931 0.1863999 6.6709868 6.6709868 2.0529013 1.3403831 1.3403831
  [330] 0.1863999 6.6709868 0.2552007 2.0529013 0.2552007 0.2417931 6.6709868
  [337] 2.0529013 0.2417931 2.0529013 0.2552007 1.3403831 2.0529013 1.3403831
  [344] 2.9172096 0.1863999 0.2417931 2.0529013 0.2417931 1.3403831 2.0529013
  [351] 0.2552007 6.6709868 1.3403831 6.6709868 2.0529013 2.9172096 0.2417931
  [358] 1.3403831 2.052901

In [14]:
s <- c(1, 2, 3)
tail(s, length(s) - 1)

[1] 2 3

Реализуем round robin.

In [31]:
time <- 0

task_schedule <- programs

while (length(task_schedule) > 0) {
    time <- time + q
    task_schedule[1] <- task_schedule[1] - q

    if (task_schedule[1] <= 0) {
        task_schedule <- tail(task_schedule, length(task_schedule) - 1)
    }
}

time / N

[1] 2.5781

In [ ]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

env <- simmer("SuperDuperSim")
env

In [ ]:
programs <- trajectory("programs' path") %>%
    select(paste0("doctor", 1:3), "round-robin") %>%
    seize_selected(1) %>%
    timeout(5) %>%
    release_selected(1)

In [ ]:
env %>%
    add_resource("server", 1) %>%
    add_generator("programmers", programs, function() rexp(1, k / t1))